In [1]:
# 주어진 청크를 바탕으로 사용자의 질문에 답변을 생성
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model="gpt-4o-mini")

question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "사용자의 질문에 대해 아래 context에 기반하여 답변하라.\n\n{context}", # LLM 이해도 개선을 위한 가독성 향샹
        ),
        MessagesPlaceholder(variable_name="messages")
    ]
)

document_chain = create_stuff_documents_chain(chat, question_answering_prompt)

In [ ]:
from langchain.memory import ChatMessageHistory
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
import os

load_dotenv()
OPEN_API_KEY = os.getenv("OPEN_API_KEY")

persist_directory = '../chroma_store'
embedding = OpenAIEmbeddings(model='text-embedding-3-large', api_key=OPEN_API_KEY)

# 벡터 db 로딩
print("Loading existing Chroma store")
vectorstore = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)

# 벡터 db에서 관련 청크 3개 가져오기
retriever = vectorstore.as_retriever(k=3)
docs = retriever.invoke("서울시의 온실가스 저감 정책에 대해 알려줘.")

# 사용자 질문 hitory에 입력
chat_history = ChatMessageHistory()
chat_history.add_user_message("서울시의 온실가스 저감 정책에 대해 알려줘.")

answer = document_chain.invoke(
    {
        "messages": chat_history.messages,
        "context": docs,
    }
)

chat_history.add_ai_message(answer)

print(answer)

Loading existing Chroma store
서울시는 탄소중립을 목표로 온실가스 저감 정책을 적극 추진하고 있습니다. 주요 내용은 다음과 같습니다:

1. **감축 목표 설정**: 서울시는 2050년까지 탄소중립을 목표로 하며, 이를 위해 2030년까지 2005년 대비 온실가스를 40%, 2040년까지 70% 감축할 계획을 세웠습니다.

2. **에너지 효율화 및 제로에너지 건물**: 건물 부문에서의 온실가스 배출량이 90%를 차지하고 있기 때문에, 2026년까지 100만 호의 건물 에너지 효율화 사업을 추진하고, 신규 건물에 대해 제로에너지 건물(ZEB) 의무화를 추진하고 있습니다.

3. **전기차 확대 및 충전 인프라 구축**: 수송 부문에서도 온실가스 배출을 줄이기 위해 2026년까지 전기차 비중을 10%로 늘리고, 22만 개의 충전 인프라를 구축할 계획입니다.

4. **기후위기 대응 전략**: 기후변화로 인한 위험을 고려하여 극한 기후 현상에 대응할 수 있는 방안을 마련하고 있으며, 시민의 생활과 안전을 보호하기 위한 다양한 대책을 개발하고 있습니다.

5. **기후예산제**: 정부의 모든 예산 사업에 온실가스 감축 방안을 고려하여 기후예산제를 시행하고 있습니다.

서울시는 이러한 정책들을 통해 도심의 지속가능성과 시민들의 건강을 증진시키는 방향으로 온실가스 저감에 힘쓰고 있습니다.


In [3]:
for m in chat_history.messages:
    print(m)

content='서울시의 온실가스 저감 정책에 대해 알려줘.' additional_kwargs={} response_metadata={}
content='서울시는 탄소중립을 목표로 온실가스 저감 정책을 적극 추진하고 있습니다. 주요 내용은 다음과 같습니다:\n\n1. **감축 목표 설정**: 서울시는 2050년까지 탄소중립을 목표로 하며, 이를 위해 2030년까지 2005년 대비 온실가스를 40%, 2040년까지 70% 감축할 계획을 세웠습니다.\n\n2. **에너지 효율화 및 제로에너지 건물**: 건물 부문에서의 온실가스 배출량이 90%를 차지하고 있기 때문에, 2026년까지 100만 호의 건물 에너지 효율화 사업을 추진하고, 신규 건물에 대해 제로에너지 건물(ZEB) 의무화를 추진하고 있습니다.\n\n3. **전기차 확대 및 충전 인프라 구축**: 수송 부문에서도 온실가스 배출을 줄이기 위해 2026년까지 전기차 비중을 10%로 늘리고, 22만 개의 충전 인프라를 구축할 계획입니다.\n\n4. **기후위기 대응 전략**: 기후변화로 인한 위험을 고려하여 극한 기후 현상에 대응할 수 있는 방안을 마련하고 있으며, 시민의 생활과 안전을 보호하기 위한 다양한 대책을 개발하고 있습니다.\n\n5. **기후예산제**: 정부의 모든 예산 사업에 온실가스 감축 방안을 고려하여 기후예산제를 시행하고 있습니다.\n\n서울시는 이러한 정책들을 통해 도심의 지속가능성과 시민들의 건강을 증진시키는 방향으로 온실가스 저감에 힘쓰고 있습니다.' additional_kwargs={} response_metadata={}
